# Books Recommender System using clustering | Collaborative Based

In [73]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
## Reading the Data
# As dataset is difficult to read because data is separated by semicolons. So, will be mentioning the seperator.
# In the dataset, there are bad lines. Dataset cannot be opened directly as there will be some kind of errors while reading the data. so using 'on_bad_lines'
# Dataset is formatted with the latin one so using encoding as 'latin-1'

books = pd.read_csv('data/BX-Books.csv', sep=";", error_bad_lines=False, warn_bad_lines=True, encoding='latin-1')




b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
c:\Users\Saurabh Kamal\Desktop\Data\Krish\Deployment\MLOPS\Batch\Book-Reco

In [8]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [9]:
books.shape

(271360, 8)

In [10]:
books.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [11]:
books = books[['ISBN','Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher','Image-URL-L']] 

In [12]:
books.head(2)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...


In [13]:
books.rename(columns={
    "Book-Title": "title",
    "Book-Author": "author",
    "Year-Of-Publication": "year",
    "Publisher": "publisher",
    "Image-URL-L": "img_url"}, inplace = True)

In [14]:
books.head()

,ISBN,title,author,year,publisher,img_url
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...


In [16]:
users = pd.read_csv('data/BX-Users.csv', sep=";", error_bad_lines=False, warn_bad_lines=True, encoding='latin-1')
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [17]:
users.shape

(278858, 3)

In [18]:
ratings = pd.read_csv('data/BX-Book-Ratings.csv', sep=";", error_bad_lines=False, warn_bad_lines=True, encoding='latin-1')
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [19]:
ratings.shape

(1149780, 3)

In [20]:
print(books.shape)
print(users.shape)
print(ratings.shape)

(271360, 6)
(278858, 3)
(1149780, 3)


In [21]:
ratings.rename(columns={
    "User-ID": "user_id",
    "Book-Rating": "rating"}, inplace=True)

In [22]:
ratings.head()

,user_id,ISBN,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [23]:
ratings['user_id'].value_counts()

11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
158698        1
17920         1
277135        1
275086        1
187812        1
Name: user_id, Length: 105283, dtype: int64

The user_id '11676' has rated 13602 books and the user_id '198711' has rated 7550 books.

A total of 1149780 users have rated this many books.

There are users who have rated only once and this kind of information is not required because this 'one' rating can't help in increasing the accuracy.
So, I will filter out the users.

In [24]:
ratings['user_id'].unique().shape

(105283,)

In [25]:
x = ratings['user_id'].value_counts() > 200

# Those users I am going to take who have rated more than 200 books

In [26]:
x

11676      True
198711     True
153662     True
98391      True
35859      True
          ...  
158698    False
17920     False
277135    False
275086    False
187812    False
Name: user_id, Length: 105283, dtype: bool

In [27]:
x[x].shape

(899,)

Out of 105283 users only 899 users have rated more than 200 books

In [28]:
y = x[x].index
y

Int64Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352,
            110973, 235105,
            ...
            260183, 155916,  44296,  73681,  59727,  28634, 188951,   9856,
            268622, 274808],
           dtype='int64', length=899)

These are the indexes of the users in the dataset. By using this index, users will be filtered out.

In [29]:
# isin(y) --> this will try to figure out those users(indexes) who have rated more than 200 books 

ratings = ratings[ratings['user_id'].isin(y)]

In [30]:
ratings.head()

,user_id,ISBN,rating
1456,277427,002542730X,10
1457,277427,0026217457,0
1458,277427,003008685X,8
1459,277427,0030615321,0
1460,277427,0060002050,0


These are the users who have rated more than 200 books

In [31]:
ratings.shape

(526356, 3)

These are the users who have rated more than 200 books.

Now, I will be merging these ratings with our books because I am figuring out books that are related to these ratings. I have user_id and the ratings and I have to figure out which books are rated 10.

In other words, want to filter out those books rated more than 200 times.

In [32]:
books.head(2)

,ISBN,title,author,year,publisher,img_url
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...


In [33]:
ratings_with_books = ratings.merge(books, on = "ISBN")

In [34]:
ratings_with_books.head()

,user_id,ISBN,rating,title,author,year,publisher,img_url
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...


These are the books that got more than 200 times ratings. 

In [35]:
ratings_with_books.shape

(487671, 8)

There are some missing values that got rid off.

Now, will be checking which books got how many type of ratings.

In [36]:
num_rating = ratings_with_books.groupby('title')['rating'].count().reset_index() 

In [37]:
num_rating.head()

,title,rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


This dataframe shows which specific book got how much rating.

In [38]:
num_rating.rename(columns={"rating" : "num_of_rating"}, inplace=True)

In [39]:
num_rating.head()

,title,num_of_rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


In [40]:
ratings_with_books.head(2)

,user_id,ISBN,rating,title,author,year,publisher,img_url
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...


In [41]:
final_rating = ratings_with_books.merge(num_rating, on='title')

In [42]:
final_rating.head(2)

,user_id,ISBN,rating,title,author,year,publisher,img_url,num_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82


Filtering out those books that have got more than 50 ratings.

In [43]:
final_rating.shape

(487671, 9)

In [44]:
final_rating = final_rating[final_rating['num_of_rating'] >= 50]

These are the books which has got more than 50 ratings.

In [45]:
final_rating.head()

,user_id,ISBN,rating,title,author,year,publisher,img_url,num_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82


These are some books which have got more than 50 ratings.

In [46]:
final_rating.sample(10)

,user_id,ISBN,rating,title,author,year,publisher,img_url,num_of_rating
22972,126604,0140244824,0,Songs in Ordinary Time (Oprah's Book Club (Pap...,Mary McGarry Morris,1996,Penguin Books,http://images.amazon.com/images/P/0140244824.0...,95
8403,209373,0446604801,0,Jack &amp; Jill (Alex Cross Novels),James Patterson,1997,Warner Books,http://images.amazon.com/images/P/0446604801.0...,77
136978,234828,0671039725,0,Carrie,Stephen King,2002,Pocket,http://images.amazon.com/images/P/0671039725.0...,57
79590,114414,0060915544,0,The Bean Trees,Barbara Kingsolver,1989,Perennial,http://images.amazon.com/images/P/0060915544.0...,132
66138,177072,0142001430,0,Year of Wonders,Geraldine Brooks,2002,Penguin Books,http://images.amazon.com/images/P/0142001430.0...,59
82175,110912,0553275976,0,The Bonfire of the Vanities,Tom Wolfe,1988,Bantam Books,http://images.amazon.com/images/P/0553275976.0...,92
132704,273979,0345441036,0,Black House,Stephen King,2002,Ballantine Books,http://images.amazon.com/images/P/0345441036.0...,70
185490,265313,0671011367,0,Deja Dead,Kathy Reichs,1998,Pocket,http://images.amazon.com/images/P/0671011367.0...,69
5866,232131,0425158616,0,Cause of Death,Patricia Daniels Cornwell,1997,Berkley Publishing Group,http://images.amazon.com/images/P/0425158616.0...,121
40337,138844,0451410491,0,Insomnia,Robert Westbrook,2002,Onyx Books,http://images.amazon.com/images/P/0451410491.0...,105


This is my final dataframe

In [47]:
final_rating.shape

(61853, 9)

Now, there are 61853. We can say that other books were not carrying relevant information. 

Here, there will be some duplicate information

In [48]:
# Check if there are duplicate values in each column of the DataFrame
def check_duplicate_values(df):
    duplicate_columns = {}
    for column in df.columns:
        duplicate_columns[column] = df[column].duplicated().any()
    return duplicate_columns


duplicate_columns = check_duplicate_values(final_rating)
print("Columns with duplicate values:", duplicate_columns)



Columns with duplicate values: {'user_id': True, 'ISBN': True, 'rating': True, 'title': True, 'author': True, 'year': True, 'publisher': True, 'img_url': True, 'num_of_rating': True}


In [49]:
final_rating.drop_duplicates(['user_id', 'title'], inplace=True)

In [50]:
final_rating.shape

(59850, 9)

duplicate information has been removed.

In [51]:
final_rating.head()

,user_id,ISBN,rating,title,author,year,publisher,img_url,num_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82


In [52]:
book_pivot = final_rating.pivot_table(columns='user_id', index='title',values='rating')

In [53]:
book_pivot

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1st to Die: A Novel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2nd Chance,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN
4 Blondes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84 Charing Cross Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,7.0,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
You Belong To Me,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [54]:
book_pivot.shape

(742, 888)

There are total 742 books and 888 user's information.

In [55]:
book_pivot.fillna(0, inplace=True)

In [56]:
book_pivot

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84 Charing Cross Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Now, all the NaN values are replace with 0.

If I pass this entire dataset to my algorithms. It will calculate all the values because if you see 0, it contains nothing, and if it is trying to calculate the distance from one column to another column, it will take a lot of time because, at the end, the clustering algorithm tries to calculate the distances. Here, you subtract 0 - 0, you will get 0. We don't want to calculate this 0 because 0 contains nothing. In the end, we have to consider those values that have some kind of quantity. 

So, we will perform csr matrix on top of that. Automatically 0 won't be considered. Because whenever it will calculate the distance with respect to all the columns and rows. It will ignore that 0 and consider those values that have some quantity. 

In [57]:
from scipy.sparse import csr_matrix

In [58]:
book_sparse = csr_matrix(book_pivot)

In [59]:
book_sparse

<742x888 sparse matrix of type '<class 'numpy.float64'>'
	with 14942 stored elements in Compressed Sparse Row format>

Passing it into the algorithm

In [60]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm = 'brute')

In [61]:
# not going to pass book_pivot because it is containing 0. I want to pass without 0.
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [62]:
distance, suggestion = model.kneighbors(book_pivot.iloc[237,:].values.reshape(1, -1), n_neighbors=6)

In [63]:
 distance

array([[ 0.        , 68.78953409, 69.5413546 , 72.64296249, 76.83098333,
        77.28518616]])

it is trying to calculate the distance with itself, that's why the first one is 0.

The above distances are 6 books it is recommending because they are similar with each other. 

In [64]:
suggestion

array([[237, 240, 238, 241, 184, 536]], dtype=int64)

These are indexes that are recommended

In [65]:
for i in range(len(suggestion)):
    print(book_pivot.index[suggestion[i]])

Index(['Harry Potter and the Chamber of Secrets (Book 2)',
       'Harry Potter and the Prisoner of Azkaban (Book 3)',
       'Harry Potter and the Goblet of Fire (Book 4)',
       'Harry Potter and the Sorcerer's Stone (Book 1)', 'Exclusive',
       'The Cradle Will Fall'],
      dtype='object', name='title')


I am getting all the recommendation. I have pass index 237, which is index of harry potter books, and it is recommending harry potter related books. 

In [66]:
# checking the title of the books
book_pivot.index[237]

'Harry Potter and the Chamber of Secrets (Book 2)'

In [67]:
book_pivot.index[3]

'4 Blondes'

That's we can indentify the books and pass it.

In [68]:
# Now, I will be keeping my books name. It will help to create a web application. So, all the books name will be stored in the variable and saving it in the pickle format.
books_name = book_pivot.index

In [70]:
books_name

Index(['1984', '1st to Die: A Novel', '2nd Chance', '4 Blondes',
       '84 Charing Cross Road', 'A Bend in the Road', 'A Case of Need',
       'A Child Called \It\": One Child's Courage to Survive"',
       'A Civil Action', 'A Cry In The Night',
       ...
       'Winter Solstice', 'Wish You Well', 'Without Remorse',
       'Wizard and Glass (The Dark Tower, Book 4)', 'Wuthering Heights',
       'Year of Wonders', 'You Belong To Me',
       'Zen and the Art of Motorcycle Maintenance: An Inquiry into Values',
       'Zoya', '\O\" Is for Outlaw"'],
      dtype='object', name='title', length=742)

In [81]:
import pickle
pickle.dump(model, open('artifacts/model.pkl', 'wb'))
pickle.dump(books_name, open('artifacts/books_name.pkl', 'wb'))
pickle.dump(final_rating, open('artifacts/final_rating.pkl', 'wb'))
pickle.dump(book_pivot, open('artifacts/book_pivot.pkl', 'wb'))


FileNotFoundError: [Errno 2] No such file or directory: 'artifacts/model.pkl'

In [82]:
np.where(book_pivot.index == "2nd Chance")[0][0]

2

In [75]:
def recommend_book(book_name):
    book_id = np.where(book_pivot.index == book_name)[0][0]
    distance, suggestion = model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1, -1), n_neighbors=6)

    for i in range(len(suggestion)):
        books = book_pivot.index[suggestion[i]]
        for j in books:
            print(j)

In [76]:
book_name = 'A Bend in the Road'
recommend_book(book_name)

A Bend in the Road
Exclusive
The Cradle Will Fall
No Safe Place
Family Album
Lake Wobegon days


In [77]:
book_pivot.index[237]

'Harry Potter and the Chamber of Secrets (Book 2)'

In [78]:
np.where(book_pivot.index == "Harry Potter and the Chamber of Secrets (Book 2)")[0][0]

237

In [79]:
book_name = 'Harry Potter and the Chamber of Secrets (Book 2)'
recommend_book(book_name)

Harry Potter and the Chamber of Secrets (Book 2)
Harry Potter and the Prisoner of Azkaban (Book 3)
Harry Potter and the Goblet of Fire (Book 4)
Harry Potter and the Sorcerer's Stone (Book 1)
Exclusive
The Cradle Will Fall


In [1]:
import ssl
print(ssl.OPENSSL_VERSION)

OpenSSL 1.1.1w  11 Sep 2023


In [2]:
import scipy
print(scipy.__version__)


1.7.3


In [3]:
# test_script.py
import scipy
import numpy as np
import pandas as pd
import streamlit as st

st.write("SciPy version:", scipy.__version__)
st.write("NumPy version:", np.__version__)
st.write("Pandas version:", pd.__version__)


2024-06-13 18:50:28.503 
  command:

    streamlit run c:\Users\Saurabh Kamal\Desktop\Data\Krish\Deployment\MLOPS\Batch\Book Recommender System\env\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
